In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline

C:\Users\ghass\anaconda\envs\LLM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Tous les modèles utilisés avec les 2 prompts pour la classification + les 2 models (minitron , sablem zephyr ) pour la summarization sont réalisés sur Nvidia RTX 4070 Nvidia GPU (8 gega dedié + 8 gega ) et 16 go (cpu) et intel 12ème génération . les autres sont exécutées dans colab.
## les modèles ont généralement pris max 5 minutes (pour phi3,H2O) pour classification et 20 (pour Sablem model voir 30 a 40 de minutes pour Minitron,Sablem 
## Les taches de classification sont plus rapides que summarization car necessite moins de génération de nombre de tokens pour la réponse

In [4]:
import torch
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    torch_dtype="auto",
    trust_remote_code=True,
).to("cuda")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.61it/s]


In [5]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    torch_dtype=torch.bfloat16,
)

Device set to use cuda:0


In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("C:/Users/ghass/Downloads/1000_enron_spam_data.csv")
df

,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,Date
0,0,6568,kirstee ' s role in london,vince :\nthis is precisely the concern i have ...,ham,2000-07-25
1,1,9797,i never sent you this - keep it hush hush ! : ...,online credit breakthroughrepair your credit o...,spam,2002-08-02
2,2,655,copanos changes,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,2000-04-12
3,3,8206,"hrgovcic , hrvoje",please incease the bonus for hrgovcic in vince...,ham,2001-01-13
4,4,25787,a chance to get new logo now,working on your company ' s image ? start with...,spam,2005-06-27
...,...,...,...,...,...,...
995,995,29351,can you help ! ! !,from : mrs . victoria ndlovu\nattn :\nyou may ...,spam,2004-08-13
996,996,32821,dont let age deter you from great sex !,20 minutes is all it takes\nmore info here . ....,spam,2005-05-13
997,997,14476,proposed retention bonuses for netco,"louise ,\ni am sorry to bother you at such a c...",ham,2001-12-28
998,998,21733,epson inkjet cartridges from 5 . 99,"save up to 75 % on inkjet , laser copier suppl...",spam,2005-04-18


In [8]:
df = df[:110]

In [9]:
spam_count = (df['Spam/Ham'] == 'spam').sum()
ham_count = (df['Spam/Ham'] == 'ham').sum()

print(f"Spam count: {spam_count}")
print(f"Ham count: {ham_count}")

Spam count: 52
Ham count: 58


In [11]:
generation_args = {
    "max_new_tokens": 350,
    "return_full_text": False,
    "temperature": 0.3,
    "do_sample": True,
}

In [12]:
import pprint
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    # Prepare the message
    text_template = """Classify the text into Spam or Ham. Please provide me at the end of the response with Class of the email:. Text: {} Class of the email:?""".format(row["Message"])

    # Set up the messages for the assistant
    messages = [
        {"role": "system", "content": "You are a clever assistant which helps answering general questions and also math and logic questions."},
        {"role": "user", "content": text_template},
    ]

    # Call your pipe function (replace `pipe` and `generation_args` with actual implementation)
    output = pipe(messages, **generation_args)

    # Extract the generated text
    generated_text = output[0]["generated_text"]
    print("example {} : \n".format(index))
    pprint.pprint(generated_text)
    print("\n")

    # Extract the class of the email
    if "Class of the email:" in generated_text:
        # Extract the portion of the text after "Class of the email:"
        email_class = generated_text.split("Class of the email:")[-1].strip().split()[0]  # Take the first word

    pprint.pprint(email_class)
    print("\n")
    #In our prompt if we write it spam or ham in the response it didn't give the class so we used instead "Spam" or "Ham" to recognize it and we formatted it to make it like the true labels
    if email_class == "Spam" or email_class == "Spam." :
        email_class = "spam"
    elif email_class == "Ham":
        email_class = "ham"

    # Print the actual Spam/Ham label from the DataFrame for comparison
    actual_class = row["Spam/Ham"]
    pprint.pprint(actual_class)


    # Store the true and predicted labels
    true_labels.append(actual_class)
    predicted_labels.append(email_class)


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


example 0 : 

(' Class of the email: Ham\n'
 '\n'
 'The text is a personal email exchange between two colleagues discussing '
 'professional concerns and seeking advice. It does not contain any '
 'promotional content, spammy links, or requests for personal information, '
 'which are common characteristics of spam emails.')


'Ham'


'ham'
example 1 : 

(' Class of the email: Spam\n'
 '\n'
 'The text provided appears to be promoting a service that claims to repair '
 'credit online, which is a common characteristic of spam emails. The use of '
 'phrases like "online credit breakthrough" and "clear up bad credit online" '
 'along with the urgency to click for more information are typical spam '
 'tactics. Additionally, the random combination of numbers and letters at the '
 'beginning of the text also suggests that it is likely spam.')


'Spam'


'spam'
example 2 : 

(' Class of the email: Spam\n'
 '\n'
 'The text provided appears to be a forwarded email with a subject line '
 'related 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


example 9 : 

(' Class of the email: Ham\n'
 '\n'
 'The text appears to be a legitimate promotional message from a '
 'pharmaceutical company offering generic medications at affordable prices. It '
 'does not exhibit characteristics typically associated with spam, such as '
 'unsolicited offers, requests for personal information, or aggressive sales '
 'tactics.')


'Ham'


'spam'
example 10 : 

' Class of the email: Spam'


'Spam'


'spam'
example 11 : 

(' Class of the email: Ham\n'
 '\n'
 'The email is a legitimate communication between colleagues discussing '
 'scheduling and meeting arrangements. It does not contain any promotional '
 'content, spammy links, or requests for personal information.')


'Ham'


'ham'
example 12 : 

(' Class of the email: Spam\n'
 '\n'
 "The text appears to be a typical spam email. It's attempting to solicit the "
 "recipient's attention by warning them about account suspension if they don't "
 'update their information. It also provides a link to a no

In [13]:
label_mapping = {'ham': 0, 'spam': 1}
true_labels = [label_mapping[label] for label in true_labels]
predicted_labels = [label_mapping[label] for label in predicted_labels]

f1 = f1_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
accuracy = accuracy_score(true_labels, predicted_labels)

In [14]:

# Print the metrics
metrics = {
    "F1 Score": f1,
    "Precision": precision,
    "Recall": recall,
    "Accuracy": accuracy
}

pprint.pprint(metrics)
print("\n")

{'Accuracy': 0.8090909090909091,
 'F1 Score': 0.8235294117647058,
 'Precision': 0.7313432835820896,
 'Recall': 0.9423076923076923}




### Comparaison des prompts pour Phi3_instruct_128k (Type et Performance) ###

# 1ère formulation :
Séparation des roles dans le prompt: 
 - Pour le système donner la tache a effecturer puis enfin  spécifier de mettre à la fin la réponse en donnant la forme de réponse afin de faciliter après le praitraitement.
 - Pour le user role: on donne le texte de l'email à traiter pour en effectuer la classification et on donne la forme de la réponse pour nous aider à l'extraire après avec le postraitement.
     
       text_template = """{} Class of the email:?""".format(row["Message"])

       messages = [
        {"role": "system", "content": "Classify the text into Spam or Ham. Please provide me at the end of the response with Class of the email:. "},
        {"role": "user", "content": text_template},
        ]
# 2ème formulation :

    text_template = """Classify the text into Spam or Ham. Please provide me at the end of the response with Class of the email:. Text: {} Class of the email:?""".format(row["Message"])

    messages = [
        {"role": "user", "content": text_template},
    ]
Utilisation d'un seule role ici la différence est que la tache a accomplir , la forme de retour et le texte à traiter sont faits sans différenciation de role.

# Tableau comparatif des performances
| Formulation | Accuracy | F1 Score | Precision | Recall |
|-------------|----------|----------|-----------|--------|
| **1**       | 85.32%   | 85.45%   | 81.03%    | 90.38% |
| **2**       | 80.91%   | 82.35%   | 73.13%    | 94.23%  |


  # => 
  La 1ère formulation est meuilleur en terme d'équilibre entre la précision et le recall puisque elle présente un bon F1 score 67.47% proche de 70% alors que 2ème formulation présente un faible F1 score .
  Cependant l'accuracy de la 2ème formulation est meuilleur que celle de la 1ère formulation mais les 2 restes moins bon .
  Donc on peut choisir la 1ère formulation.


## Raison de bonne performance
Le modèle donne des explications et non pas seulement la réponse comme dans le cas de l'exemple 5 (pour le spam)

## Analyse des erreurs
example 59 , 82(prompt1) pas de présence d'explication c'est difficile de trouver la raison
example 71(prompt1) car on a un aspect commercial dans le contenu d'ou la confusion avec  les spam alors qu'il est ham
example 58(prompt2) utilisation de termes incohérents cad ici la personne qui a écrit l'email l'a mal formulé d'ou la confusion d'etre spam au lieu de ham

###  Conclusion 
- Le modèle Phi 3 a prouvé une meuilleure performance parmi les modèles dans - - les 2 taches surtout la classification.
- Le modèle H2O est moins bon , aussi Sablem pour la classification et aussi le résumé
- Le modèle Minitron est mauvais pour les taches de classification et résumé 
